# Purpose
1. 請為客戶進行貼標，並依其特性進行命名，以幫助行銷部門推出新產品和制定行銷方式。
2. 請先進行降維後再分群(依你的判斷選擇降維和分群的方法)

In [ ]:
import pandas as pd

# 1. import data

In [ ]:
df = pd.read_csv('segmentation_data.csv')
df.head()

In [ ]:
df.info()

#### ID：顯示客戶的唯一標識。

#### Sex：
0：男性        
1：女性        

#### Marital status：客戶的婚姻狀況。
0：單人        
1：非單身（離婚/分居/已婚/喪偶）        

#### Age：客戶的年齡（以年為單位），計算方法為當年減去創建數據集時客戶的出生年份
18 最小值（數據集中觀察到的最低年齡）        
76 最大值（數據集中觀察到的最高年齡）        

#### Education：客戶的教育水準。
0：其他/未知        
1：高中        
2：大學        
3：研究生院        

#### Income：客戶以美元自我報告的年收入。
35832 最小值（數據集中觀察到的最低收入）        
309364 最大值（數據集中觀察到的最高收入）        

#### Occupation ：客戶的職業類別。
0：失業 / unskilled        
1：skilled employee / official        
2：management / self-employed / highly qualified employee / officer        

#### Settlement size：客戶居住城市的規模。
0：小城市        
1：中等城市        
2：大城市        

In [ ]:
df.describe().round(2).reset_index(level=0)

In [ ]:
# 檢查na 值
df.isnull().sum()
# 如果有na值 df.dropna(inplace = True)

In [ ]:
# 性別總數
df.groupby(by="Sex",as_index=False).size().rename({'size':'cnt'}, axis=1)

In [ ]:
# 現在是否有職業
df.groupby(by="Settlement size",as_index=False).size().rename({'size':'cnt'}, axis=1)

# 資料前處理

In [ ]:
df.head()

In [ ]:
# drop 掉id
df_new = df.drop(columns=["ID"])
df_new.head()

In [ ]:
#標準化
from sklearn.preprocessing import StandardScaler

# 將類型以外的 11 個特徵全取出
X = df_new.iloc[:, 0:] 

# 使用 scikit-learn 內建的 API 標準化
scaler = StandardScaler()
Z_sk = scaler.fit_transform(X)

In [ ]:
# 更新 DataFrame
df_new.iloc[:, 0:] = Z_sk
df_new.head()

In [ ]:
# 顯示各特徵的平均與標準差
df_stats = df_new.describe().loc[['mean', 'std']]
df_stats.round(2)

# Modeling
- 降維
  - PCA
  - MDS (尚未寫)
- 分群
  - k-means


## PCA

In [ ]:
# 做七個主成分的時候 解釋力從1到7排列
from sklearn.decomposition import PCA
import numpy as np

pca_7d = PCA(7, random_state=123)
pca_7d.fit(Z_sk)
np.round(pca_7d.explained_variance_ratio_, 2)

In [ ]:
print(f'前2個主成分解釋力達: {sum(pca_7d.explained_variance_ratio_.tolist()[:2])}') 
print(f'前3個主成分解釋力達: {sum(pca_7d.explained_variance_ratio_.tolist()[:3])}') 

In [ ]:
import matplotlib.pyplot as plt
# 第一個位置為第一主成份的變異，所以要加0在第一個位置
var_plot = [0] + pca_7d.explained_variance_ratio_.tolist()

cum_explained_var_ratio = np.cumsum(var_plot)
plt.plot(cum_explained_var_ratio)
plt.xlabel('# principal components')
plt.ylabel('cumulative explained variance');

In [ ]:
# 主成分分析其實就是在分析共變異數矩陣(covariance matrix)，矩陣的特徵向量(eigenvector)就是主成分，特徵值(eigenvalue)由大到小依序排列就是第一主成分、第二主成分...。
# 依照eigenvalue的大小排序共變異數矩陣的eigenvectors
pca = PCA(n_components=2, random_state=123)

# 注意我們是對標準化後的特徵做PCA
pca_trans = pca.fit_transform(Z_sk)

In [ ]:
# 檢視2個主成分跟原本特徵之關係
# 可以為主成分命名 變成對所有特徵突出的做新的綜合特徵
pcs = np.array(pca.components_)

df_pc = pd.DataFrame(pcs, columns=df.columns[1:])
df_pc.index = [f"第{c}主成分" for c in['一', '二']]
df_pc.style\
    .background_gradient(cmap='PuBu', axis=None)\
    .format("{:.2}")

In [ ]:
# 第一主成分顯示在 x 軸，第二主成分在 y 軸
plt.scatter(pca_trans[:, 0], pca_trans[:, 1])
plt.axis('equal');

In [ ]:
# 把主成分 concat回原本的資料表

# 興建df
factor_tb = pd.DataFrame(pca_trans, columns=['factor1', 'factor2'])

df_factor_tb = pd.concat([df_new.reset_index(drop=True), factor_tb], axis=1)
df_factor_tb

In [ ]:
# 男女在主成分分析的圖表化
# 可以為主成分命名 變成對所有特徵突出的做新的綜合特徵 畫圖時可以用男女做解釋在主成分中的定位
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
%matplotlib inline

print('PCA visualization:')
sns.lmplot(data=df_factor_tb, x='factor1', y='factor2', hue='Sex', 
                   fit_reg=False, legend=True, legend_out=True)

## K-means
- 手軸法
- 輪廓分析法

In [ ]:
from sklearn.cluster import KMeans

# 用手軸法找最適合分幾群 


clusters=list(range(2,8))  #2-8群
ssd = []
for num_clusters in clusters:
    model_clus = KMeans(n_clusters = num_clusters, max_iter=150,random_state= 50)
    model_clus.fit(Z_sk)
    ssd.append(model_clus.inertia_)

plt.plot(clusters, ssd);

In [ ]:
# 輪廓分析法
from sklearn.metrics import silhouette_score

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ,13, 14 ]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50,random_state= 100)
    kmeans.fit(Z_sk)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(Z_sk, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))




 (silhouette score越大越好)

 因此選12群

In [ ]:
cluster = KMeans(n_clusters = 12, random_state = 111)
cluster.fit(Z_sk)
y_kmeans = cluster.predict(Z_sk)
print(y_kmeans)

In [ ]:
#count number of records in every cluster
pd.Series(cluster.labels_).value_counts()

檢視分群結果

In [ ]:
# 把k-means結果做成df 
pred = pd.DataFrame(cluster.labels_, columns=['k_cluster'])
# concat回原本的df
kmeans_result = pd.concat([df.reset_index(drop=True), pred], axis=1)
kmeans_result

In [ ]:
# 群類=4的data
kmeans_result[kmeans_result['k_cluster'] == 4].head()

In [ ]:
# 12群的平均
kmeans_result.groupby(['k_cluster'], as_index=True).mean()

### 劃出散佈圖